In [1]:
%load_ext autoreload
%autoreload 2

import io
import tarfile
import pandas as pd
import numpy as np
from tqdm import tqdm

# Suppress warning for xlrd
import warnings

# Filter all warning that start with "WARNING"
warnings.filterwarnings('ignore')


In [2]:
import time
import tarfile


In [16]:
output_tarfile = 'formula2.tar'
output_tar_stream = tarfile.open(output_tarfile, mode='w|')

In [17]:
input_file = '/Users/dirkocoetsee/Downloads/fuse-binaries-dec2014.tar.gz'

file = input_file
tar = tarfile.open(file, mode='r|*')
maxrows = 0
maxcols = 0

words = []
t0 = time.time()
total_time_open = 0
icounter = 0
for fcounter, member in enumerate(tar):
    t0_loop = time.time()
    if fcounter > 1_000_000:
        break
    if not member.isfile():
        continue
    try:
        buffer = tar.extractfile(member)
        buffer_seekable = io.BytesIO(buffer.read())
        dfs = pd.read_excel(buffer_seekable, sheet_name=None, header=None, index_col=None, nrows=1000)
        t1 = time.time()
        total_time_open += t1 - t0_loop
    except AttributeError:
        continue
    except IndexError:
        continue
    except Exception as e:
        continue
    contains_formula = False
    for dfi, df in enumerate(dfs.values()):
        # Iterate over all cells
        if df.shape[0] > maxrows:
            maxrows = df.shape[0]
        if df.shape[1] > maxcols:
            maxcols = df.shape[1]
        for v in df.values.flatten():
            strv = str(v)
            if strv.startswith('=') and ('!' in strv or 'LOOKUP' in strv):
                contains_formula = True
                break
        if contains_formula:
            break
    if contains_formula:
        buffer_seekable.seek(0)
        output_tar_stream.addfile(member, buffer_seekable)
        icounter += 1

    if fcounter % 1000 == 0:
        print(f'{icounter}', fcounter, '  ', len(dfs), list(dfs.values())[0].shape, (maxrows, maxcols))


0 2000    1 (73, 3) (1000, 256)
0 3000    2 (1000, 5) (1000, 256)
0 4000    1 (69, 3) (1000, 256)
0 6000    1 (2, 10) (1000, 16384)
0 7000    3 (261, 58) (1000, 16384)
1 9000    1 (84, 3) (1000, 16384)
1 12000    1 (6, 8) (1000, 16384)
1 13000    1 (64, 7) (1000, 16384)
1 14000    1 (0, 0) (1000, 16384)
1 15000    3 (261, 58) (1000, 16384)
1 16000    1 (1, 13) (1000, 16384)
1 17000    3 (261, 58) (1000, 16384)
1 20000    2 (155, 3) (1000, 16384)
1 21000    1 (1, 13) (1000, 16384)
1 23000    1 (19, 3) (1000, 16384)
1 24000    1 (1, 13) (1000, 16384)
1 25000    3 (70, 8) (1000, 16384)
1 26000    1 (3, 13) (1000, 16384)
1 28000    1 (1, 13) (1000, 16384)
1 29000    1 (30, 25) (1000, 16384)
1 30000    2 (58, 1) (1000, 16384)
1 31000    11 (1, 5) (1000, 16384)
2 34000    1 (4, 13) (1000, 16384)
2 35000    1 (4, 25) (1000, 16384)
2 36000    1 (68, 3) (1000, 16384)
2 38000    1 (25, 43) (1000, 16384)
2 39000    1 (1, 13) (1000, 16384)
2 40000    4 (404, 13) (1000, 16384)
2 41000    1 (68, 3) 

In [7]:
1+1

2

In [14]:
input_file = 'formula.tar'

file = input_file
tar = tarfile.open(file, mode='r|*')
maxrows = 0
maxcols = 0

icounter = 0
words = []
forms = set()
t0 = time.time()
total_time_open = 0
for fcounter, member in enumerate(tar):
    t0_loop = time.time()
    if fcounter > 1_000_000:
        break
    if not member.isfile():
        continue
    try:
        buffer = tar.extractfile(member)
        buffer_seekable = io.BytesIO(buffer.read())
        dfs = pd.read_excel(buffer_seekable, sheet_name=None, header=None, index_col=None, nrows=1000)
        t1 = time.time()
        total_time_open += t1 - t0_loop
    except AttributeError:
        continue
    except IndexError:
        continue
    except Exception as e:
        continue
    for dfi, df in enumerate(dfs.values()):
        # Iterate over all cells
        if df.shape[0] > maxrows:
            maxrows = df.shape[0]
        if df.shape[1] > maxcols:
            maxcols = df.shape[1]
        for v in df.values.flatten():
            strv = str(v)
            if strv.startswith('=') and '!' in strv and strv not in forms:
                forms.add(strv)
                icounter += 1
                print(strv)
    print(f'{icounter}', fcounter, '  ', len(dfs), list(dfs.values())[0].shape, (maxrows, maxcols), len(forms))

0 0    1 (85, 8) (85, 8) 0
='Rabies form'!L75
1 1    7 (2, 130) (357, 130) 1
1 2    16 (25, 9) (357, 256) 1
1 3    17 (25, 9) (357, 256) 1
1 4    16 (25, 9) (357, 256) 1
1 5    16 (25, 9) (357, 256) 1
1 6    1 (1000, 4) (1000, 256) 1
1 7    11 (25, 12) (1000, 256) 1
1 8    1 (1000, 11) (1000, 256) 1
1 9    1 (147, 4) (1000, 256) 1
1 10    5 (14, 8) (1000, 256) 1
='H:\2013 Tax Roll\[t002.xls]TreasStlmnt'!$C$12
='H:\2013 Tax Roll\[t002.xls]TreasStlmnt'!$C$13
='H:\2013 Tax Roll\[t002.xls]TreasStlmnt'!$C$14
='H:\2013 Tax Roll\[t002.xls]TreasStlmnt'!$C$15
='H:\2013 Tax Roll\[t002.xls]TreasStlmnt'!$C$16
='H:\2013 Tax Roll\[t002.xls]TreasStlmnt'!$C$17
='H:\2013 Tax Roll\[t002.xls]TreasStlmnt'!$C$18
='H:\2013 Tax Roll\[t002.xls]TreasStlmnt'!$C$19
='H:\2013 Tax Roll\[t002.xls]TreasStlmnt'!$C$21
='H:\2013 Tax Roll\[t002.xls]TreasStlmnt'!$C$22
='H:\2013 Tax Roll\[t002.xls]TreasStlmnt'!$C$23
='H:\2013 Tax Roll\[t002.xls]TreasStlmnt'!$C$24
='H:\2013 Tax Roll\[t002.xls]TreasStlmnt'!$C$25
='H:\2013 T

ReadError: unexpected end of data

In [15]:
(forms)

{"='H:\\2013 Tax Roll\\[t002.xls]Input'!O$53",
 "='H:\\2013 Tax Roll\\[t002.xls]TreasStlmnt'!$C$12",
 "='H:\\2013 Tax Roll\\[t002.xls]TreasStlmnt'!$C$13",
 "='H:\\2013 Tax Roll\\[t002.xls]TreasStlmnt'!$C$14",
 "='H:\\2013 Tax Roll\\[t002.xls]TreasStlmnt'!$C$15",
 "='H:\\2013 Tax Roll\\[t002.xls]TreasStlmnt'!$C$16",
 "='H:\\2013 Tax Roll\\[t002.xls]TreasStlmnt'!$C$17",
 "='H:\\2013 Tax Roll\\[t002.xls]TreasStlmnt'!$C$18",
 "='H:\\2013 Tax Roll\\[t002.xls]TreasStlmnt'!$C$19",
 "='H:\\2013 Tax Roll\\[t002.xls]TreasStlmnt'!$C$21",
 "='H:\\2013 Tax Roll\\[t002.xls]TreasStlmnt'!$C$22",
 "='H:\\2013 Tax Roll\\[t002.xls]TreasStlmnt'!$C$23",
 "='H:\\2013 Tax Roll\\[t002.xls]TreasStlmnt'!$C$24",
 "='H:\\2013 Tax Roll\\[t002.xls]TreasStlmnt'!$C$25",
 "='H:\\2013 Tax Roll\\[t002.xls]TreasStlmnt'!$C$26",
 "='H:\\2013 Tax Roll\\[t002.xls]TreasStlmnt'!$C$27",
 "='H:\\2013 Tax Roll\\[t002.xls]TreasStlmnt'!$C$28",
 "='H:\\2013 Tax Roll\\[t002.xls]TreasStlmnt'!$C$29",
 "='H:\\2013 Tax Roll\\[t002.xls]Tr